# Analisis de Egresos Hospitalarios

In [1]:
import polars as pl
import glob

DICT_VARIABLES = {
    "ESTABLECIMIENTO_SALUD": pl.Int32,
    "GLOSA_ESTABLECIMIENTO_SALUD": pl.Categorical,
    "PERTENENCIA_ESTABLECIMIENTO_SALUD": pl.Categorical,
    "SEREMI": pl.Int8,
    "SERVICIO_DE_SALUD": pl.Int8,
    "SEXO": pl.Int8,
    "EDAD_CANT": pl.Int8,
    "TIPO_EDAD": pl.Int8,
    "EDAD_A_OS": pl.Int8,
    "PUEBLO_ORIGINARIO": pl.Int8,
    "PAIS_ORIGEN": pl.Int16,
    "GLOSA_COMUNA_RESIDENCIA": pl.Categorical,
    "REGION_RESIDENCIA": pl.Categorical,
    "GLOSA_REGION_RESIDENCIA": pl.Categorical,
    "PREVISION": pl.Int8,
    "BENEFICIARIO": pl.Categorical,
    "MODALIDAD": pl.Int8,
    "PROCEDENCIA": pl.Int8,
    "ANO_EGRESO": pl.Int16,
    "FECHA_EGRESO": pl.Date,
    "AREA_FUNCIONAL_EGRESO": pl.Int16,
    "DIAS_ESTADA": pl.Int16,
    "CONDICION_EGRESO": pl.Int8,
    "INTERV_Q": pl.Int8,
    "CODIGO_INTERV_Q_PPAL": pl.Int32,
    "PROCED": pl.Categorical,
    "CODIGO_PROCED_PPAL": pl.Categorical,
    "GLOSA_PROCED_PPAL": pl.Categorical,
}

HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax


In [2]:
def obtener_diagnosticos_unicos_de_hospital(df, hospital_a_analizar):
    diags_hospital = (
        df.filter(pl.col("ESTABLECIMIENTO_SALUD") == hospital_a_analizar)
        .select(pl.col("DIAG1"))
        .unique()
    )

    return diags_hospital


def obtener_metricas_egresos(df, agrupar_por):
    metricas_agregadas = df.groupby(agrupar_por).agg(
        [
            pl.col("DIAG1").count().alias("n_egresos"),
            pl.col("DIAS_ESTADA").mean().alias("dias_estada_promedio"),
            pl.col("INTERV_Q").sum().alias("n_int_q"),
            pl.col("CONDICION_EGRESO").sum().alias("n_muertos"),
        ]
    )

    return metricas_agregadas


def remapear_int_q_y_muertes(df):
    tmp = df.with_columns(
        [
            pl.col("INTERV_Q").map_dict({2: 0}, default=pl.col("INTERV_Q")).alias("INTERV_Q"),
            pl.col("CONDICION_EGRESO")
            .map_dict({1: 0, 2: 1}, default=pl.col("CONDICION_EGRESO"))
            .alias("CONDICION_EGRESO"),
        ]
    )

    return tmp

In [3]:
with pl.StringCache():
    df_nacional = pl.scan_csv("input/utf-8/*.csv", separator=";")
    diags_torax = (
        obtener_diagnosticos_unicos_de_hospital(df_nacional, HOSPITAL_A_ANALIZAR).collect(
            streaming=True
        )
    ).to_series()
    df = df_nacional.filter(pl.col("DIAG1").is_in(diags_torax))
    df = remapear_int_q_y_muertes(df)

    agrupacion_anio = [
        "ANO_EGRESO",
        "ESTABLECIMIENTO_SALUD",
        "GLOSA_ESTABLECIMIENTO_SALUD",
        "DIAG1",
    ]

    metricas = obtener_metricas_egresos(df, agrupacion_anio).collect(streaming=True)

# Analisis de Estratos

In [31]:
PERTENECE_SNSS = "Pertenecientes al Sistema Nacional de Servicios de Salud, SNSS"
NO_PERTENECE_SNSS = "No Pertenecientes al Sistema Nacional de Servicios de Salud, SNSS"
HOSPITALES_GRD = [
    118100,
    110100,
    115100,
    121117,
    103100,
    116110,
    119100,
    113100,
    114101,
    105101,
    116108,
    116105,
    101100,
    114105,
    105100,
    112102,
    133150,
    126100,
    121110,
    121114,
    129106,
    113150,
    107100,
    106100,
    113130,
    112100,
    121121,
    109100,
    106103,
    113180,
    122100,
    123100,
    107102,
    110120,
    105102,
    111101,
    111100,
    108101,
    124105,
    128109,
    109101,
    114103,
    102100,
    103101,
    120101,
    117101,
    121109,
    112101,
    104103,
    115107,
    107101,
    110130,
    116100,
    118105,
    115110,
    112103,
    104100,
    108100,
    112104,
    117102,
    106102,
    111195,
    129100,
    110150,
    125100,
]


def obtener_codigos_estratos(df_nacional, hospital_interno):
    df_publicos = df_nacional.filter(pl.col("PERTENENCIA_ESTABLECIMIENTO_SALUD") == PERTENECE_SNSS)
    df_privados = df_nacional.filter(
        (pl.col("PERTENENCIA_ESTABLECIMIENTO_SALUD") == NO_PERTENECE_SNSS)
        | (pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANALIZAR)
    )

    codigos_nacionales = (
        df_nacional.select(pl.col("ESTABLECIMIENTO_SALUD")).unique().collect(streaming=True)
    )
    codigos_publicos = (
        df_publicos.select(pl.col("ESTABLECIMIENTO_SALUD")).unique().collect(streaming=True)
    )
    codigos_privados = (
        df_privados.select(pl.col("ESTABLECIMIENTO_SALUD")).unique().collect(streaming=True)
    )

    diccionario_estratos = {
        "nacionales": codigos_nacionales.to_series(),
        "publicos": codigos_publicos.to_series(),
        "privados": codigos_privados.to_series(),
        "grd": HOSPITALES_GRD,
        "interno": [hospital_interno],
    }

    return diccionario_estratos

In [ ]:
diccionario_estratos = obtener_codigos_estratos(df_nacional, HOSPITAL_A_ANALIZAR)

In [61]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]
unir_tablas_por = agrupacion_anio


In [65]:
def obtener_metricas_para_un_estrato(
    df_estrato, glosa_estrato, variable_analisis, subgrupo_del_ranking
):
    sufijo_cols = f"_{glosa_estrato}_{variable_analisis}"
    var_rank = f"ranking{sufijo_cols}"
    var_porc = f"%{sufijo_cols}"
    var_total = f"total{sufijo_cols}"

    resumen = df_estrato.with_columns(
        (pl.col("DIAG1").cumcount().over(subgrupo_del_ranking) + 1).alias(var_rank),
        (pl.col("n_egresos").sum().over(subgrupo_del_ranking)).alias(var_total),
    )

    resumen = resumen.with_columns((pl.col("n_egresos") / pl.col(var_total)).alias(var_porc))

    return resumen


def obtener_resumen_por_estratos(df, dict_estratos, variables_a_rankear, subgrupo_del_ranking):
    for variable_analisis in variables_a_rankear:
        resultado_estrato = {}
        df = df.sort(subgrupo_del_ranking + [variable_analisis], descending=True)

        for glosa_estrato, codigos_en_estrato in dict_estratos.items():
            if glosa_estrato == "interno":
                subgrupo_del_ranking.remove("DIAG1")

            df_estrato = df.filter(pl.col("ESTABLECIMIENTO_SALUD").is_in(codigos_en_estrato))
            resumen = obtener_metricas_para_un_estrato(
                df_estrato, glosa_estrato, variable_analisis, subgrupo_del_ranking
            )

            resultado_estrato[glosa_estrato] = resumen

    return resultado_estrato

In [66]:
resultados = obtener_resumen_por_estratos(
    metricas, diccionario_estratos, variables_a_rankear, subgrupo_del_ranking
)

In [79]:
from functools import reduce


def left_join_consecutivo(left, right):
    return left.join(
        right,
        how="left",
        on=metricas.columns
    )


In [80]:
resultado = reduce(left_join_consecutivo, resultados.values())
